In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

C:\Users\avoyd\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
from tensorflow.contrib import rnn
from tensorflow.python.ops import variable_scope
from tensorflow.python.framework import dtypes
import copy

In [ ]:
#Double-layer seq2seq LSTM model
class LSTM_model():
    def __init__(self, feats_used, output_class):
        self.n_epochs = 2000
        self.batch_size = 512
        self.keep_rate = 0.5
        self.lr = 0.001
        self.feats_used = feats_used
        self.output_class = output_class
        
        self.hidden_dim = 512
        self.input_dim = len(feats_used)
        self.output_dim = 1
        self.input_seq_len = 10
        self.output_seq_len = 1
        self.num_stacked_layers = 2
        self.lambda_l2_reg = 0.003 
        self.GRADIENT_CLIPPING = 2.5
     
 
    def generate_train_samples(self, x, y, input_seq_len, output_seq_len, batch_size = 16):

        total_start_points = len(x) - input_seq_len - output_seq_len
        start_x_idx = np.random.choice(range(total_start_points), batch_size, replace = False)

        input_batch_idxs = [list(range(i, i+input_seq_len)) for i in start_x_idx]
        input_seq = np.take(x, input_batch_idxs, axis = 0)

        output_batch_idxs = [list(range(i+input_seq_len, i+input_seq_len+output_seq_len)) for i in start_x_idx]
        output_seq = np.take(y, output_batch_idxs, axis = 0)

        return (input_seq, output_seq) # in shape: (batch_size, time_steps, feature_dim)

    def generate_test_samples(self, x, y, input_seq_len, output_seq_len):

        total_samples = x.shape[0]

        input_batch_idxs = [list(range(i, i+input_seq_len)) for i in range((total_samples-input_seq_len-output_seq_len))]
        input_seq = np.take(x, input_batch_idxs, axis = 0)

        output_batch_idxs = [list(range(i+input_seq_len, i+input_seq_len+output_seq_len)) for i in range((total_samples-input_seq_len-output_seq_len))]
        output_seq = np.take(y, output_batch_idxs, axis = 0)

        return (input_seq, output_seq)
    
    def get_data(self, paths):
            
        frames_X, frames_Y = [], []
        for path in paths:
            frames_X.append(pd.read_csv(path, usecols = self.feats_used))
            frames_Y.append(pd.read_csv(path, usecols = self.output_class))
        return (pd.concat(frames_X), pd.concat(frames_Y))
    
    
    def build_seq2seq_model(self, feed_previous = False):
 
        global_step = tf.Variable(
                      initial_value=0,
                      name="global_step",
                      trainable=False,
                      collections=[tf.GraphKeys.GLOBAL_STEP, tf.GraphKeys.GLOBAL_VARIABLES])

        weights = {
            'out': tf.get_variable('Weights_out', \
                                   shape = [self.hidden_dim, self.output_dim], \
                                   dtype = tf.float32, \
                                   initializer = tf.contrib.layers.xavier_initializer())
        }
        biases = {
            'out': tf.get_variable('Biases_out', \
                                   shape = [self.output_dim], \
                                   dtype = tf.float32, \
                                   initializer = tf.zeros_initializer())
        }
 
        with tf.variable_scope('Seq2seq'):
            # Encoder: inputs
            enc_inp = [
                tf.placeholder(tf.float32, shape=(None, self.input_dim), name="inp_{}".format(t))
                   for t in range(self.input_seq_len)
            ]

            # Decoder: target outputs
            target_seq = [
                tf.placeholder(tf.float32, shape=(None, self.output_dim), name="y".format(t))
                  for t in range(self.output_seq_len)
            ]

            # Give a "GO" token to the decoder.
            # If dec_inp are fed into decoder as inputs, this is 'guided' training; otherwise only the
            # first element will be fed as decoder input which is then 'un-guided'
            dec_inp = [tf.zeros_like(target_seq[0], dtype=tf.float32, name="GO")] + target_seq[:-1]

            with tf.variable_scope('LSTMCell'):
                cells = []
                for i in range(self.num_stacked_layers):
                    with tf.variable_scope('RNN_{}'.format(i)):
                        cells.append(tf.contrib.rnn.LSTMCell(self.hidden_dim))
                cell = tf.contrib.rnn.MultiRNNCell(cells)

            def _rnn_decoder(decoder_inputs,
                            initial_state,
                            cell,
                            loop_function=None,
                            scope=None):
                """RNN decoder for the sequence-to-sequence model.
                Args:
                decoder_inputs: A list of 2D Tensors [batch_size x input_size].
                initial_state: 2D Tensor with shape [batch_size x cell.state_size].
                cell: rnn_cell.RNNCell defining the cell function and size.
                loop_function: If not None, this function will be applied to the i-th output
                  in order to generate the i+1-st input, and decoder_inputs will be ignored,
                  except for the first element ("GO" symbol). This can be used for decoding,
                  but also for training to emulate http://arxiv.org/abs/1506.03099.
                  Signature -- loop_function(prev, i) = next
                    * prev is a 2D Tensor of shape [batch_size x output_size],
                    * i is an integer, the step number (when advanced control is needed),
                    * next is a 2D Tensor of shape [batch_size x input_size].
                scope: VariableScope for the created subgraph; defaults to "rnn_decoder".
                Returns:
                A tuple of the form (outputs, state), where:
                  outputs: A list of the same length as decoder_inputs of 2D Tensors with
                    shape [batch_size x output_size] containing generated outputs.
                  state: The state of each cell at the final time-step.
                    It is a 2D Tensor of shape [batch_size x cell.state_size].
                    (Note that in some cases, like basic RNN cell or GRU cell, outputs and
                     states can be the same. They are different for LSTM cells though.)
                """
                with variable_scope.variable_scope(scope or "rnn_decoder"):
                    state = initial_state
                    outputs = []
                    prev = None
                    for i, inp in enumerate(decoder_inputs):
                        if loop_function is not None and prev is not None:
                            with variable_scope.variable_scope("loop_function", reuse=True):
                                inp = loop_function(prev, i)
                        if i > 0:
                            variable_scope.get_variable_scope().reuse_variables()
                        output, state = cell(inp, state)
                        outputs.append(output)
                        if loop_function is not None:
                            prev = output
                return (outputs, state)

            def _basic_rnn_seq2seq(encoder_inputs,
                                  decoder_inputs,
                                  cell,
                                  feed_prev,
                                  dtype=dtypes.float32,
                                  scope=None):
                """Basic RNN sequence-to-sequence model.
                This model first runs an RNN to encode encoder_inputs into a state vector,
                then runs decoder, initialized with the last encoder state, on decoder_inputs.
                Encoder and decoder use the same RNN cell type, but don't share parameters.
                Args:
                encoder_inputs: A list of 2D Tensors [batch_size x input_size].
                decoder_inputs: A list of 2D Tensors [batch_size x input_size].
                feed_previous: Boolean; if True, only the first of decoder_inputs will be
                  used (the "GO" symbol), all other inputs will be generated by the previous
                  decoder output using _loop_function below. If False, decoder_inputs are used
                  as given (the standard decoder case).
                dtype: The dtype of the initial state of the RNN cell (default: tf.float32).
                scope: VariableScope for the created subgraph; default: "basic_rnn_seq2seq".
                Returns:
                A tuple of the form (outputs, state), where:
                  outputs: A list of the same length as decoder_inputs of 2D Tensors with
                    shape [batch_size x output_size] containing the generated outputs.
                  state: The state of each decoder cell in the final time-step.
                    It is a 2D Tensor of shape [batch_size x cell.state_size].
                """
                with variable_scope.variable_scope(scope or "basic_rnn_seq2seq"):
                    enc_cell = copy.deepcopy(cell)
                    _, enc_state = rnn.static_rnn(enc_cell, encoder_inputs, dtype=dtype)
                    if feed_prev:
                        return _rnn_decoder(decoder_inputs, enc_state, cell, _loop_function)
                    else:
                        return _rnn_decoder(decoder_inputs, enc_state, cell)

            def _loop_function(prev, _):
                '''Naive implementation of loop function for _rnn_decoder. Transform prev from
                dimension [batch_size x hidden_dim] to [batch_size x output_dim], which will be
                used as decoder input of next time step '''
                return tf.matmul(prev, weights['out']) + biases['out']

            dec_outputs, dec_memory = _basic_rnn_seq2seq(
                enc_inp,
                dec_inp,
                cell,
                feed_prev = feed_previous
            )
            #print(dec_outputs)
            reshaped_outputs = [tf.matmul(i, weights['out']) + biases['out'] for i in dec_outputs]

        # Training loss and optimizer
        with tf.variable_scope('Loss'):
            # L2 loss
            output_loss = 0
            for _y, _Y in zip(reshaped_outputs, target_seq):
                output_loss += tf.reduce_mean(tf.pow(_y - _Y, 2))

            # L2 regularization for weights and biases
            reg_loss = 0
            for tf_var in tf.trainable_variables():
                if 'Biases_' in tf_var.name or 'Weights_' in tf_var.name:
                    reg_loss += tf.reduce_mean(tf.nn.l2_loss(tf_var))

            loss = output_loss + lambda_l2_reg * reg_loss
        with tf.variable_scope('Optimizer'):
            optimizer = tf.contrib.layers.optimize_loss(
                    loss = loss,
                    learning_rate=self.lr,
                    global_step = global_step,
                    optimizer='Adam',
                    clip_gradients=self.GRADIENT_CLIPPING)

        saver = tf.train.Saver
 
        return dict(
            enc_inp = enc_inp,
            target_seq = target_seq,
            train_op = optimizer,
            loss=loss,
            saver = saver,
            reshaped_outputs = reshaped_outputs,
            )

    def train(self, sess, train_paths):
        X, Y = self.get_data(train_paths) #All train data in one series
        X_train, Y_train = X.values, Y.values
        
        train_losses = []

        rnn_model = self.build_seq2seq_model(feed_previous=False)


        train_preds, train_targets = [], []

        sess.run(tf.global_variables_initializer())

        try:
            print("Training initiated...")
            for i in range(self.n_epochs): #1 iteration per epoch
                batch_input, batch_output = generate_train_samples(X_train, Y_train, 
                                                                   self.input_seq_len, self.output_seq_len,
                                                                   batch_size=self.batch_size)
                feed_dict = {rnn_model['enc_inp'][t]: batch_input[:,t] for t in range(self.input_seq_len)}
                feed_dict.update({rnn_model['target_seq'][t]: batch_output[:,t] for t in range(self.output_seq_len)})
                _, loss_t, reshaped_outputs, target_seq = sess.run([rnn_model['train_op'], rnn_model['loss'],
                                                               rnn_model['reshaped_outputs'], rnn_model['target_seq']],
                                                              feed_dict)
                print("Training iterations: {}, Loss: {}".format(i+1, loss_t))
                if (i % 10 == 0) : 
                    train_losses.append(loss_t)
                    for i in range(len(target_seq)):
                        train_preds.append(reshaped_outputs[i]) 
                        train_targets.append(target_seq[i]) 
        except KeyboardInterrupt:
            print("Training interrupted.")

        temp_saver = rnn_model['saver']()
        save_path = temp_saver.save(sess, os.path.join('./saved_model/', 'multivariate_ts_pollution_case'))
        #plt.plot(train_losses) 
        
        print("Checkpoint saved at: ", save_path)
        train_targets = [tar[0][0] for tar in train_targets]
        train_preds = [tar[0][0] for tar in train_preds]
        return (train_preds, train_targets, train_losses)
    
    def test(self, sess, test_paths):
        print("Started testing...")
        rnn_model = self.build_seq2seq_model(feed_previous=True)

        X, Y = get_data(test_paths) #All test data in one series
        X_test, Y_test = X.values, Y.values
        test_x, test_y = generate_test_samples(X_test, Y_test, input_seq_len, output_seq_len)

        sess.run(tf.global_variables_initializer())

        saver = rnn_model['saver']().restore(sess,  os.path.join('./saved_model/', 'multivariate_ts_pollution_case'))

        feed_dict = {rnn_model['enc_inp'][t]: test_x[:, t, :] for t in range(self.input_seq_len)}
        feed_dict.update({rnn_model['target_seq'][t]: np.zeros([test_x.shape[0], self.output_dim], dtype=np.float32) for t in range(self.output_seq_len)})
        final_preds = sess.run(rnn_model['reshaped_outputs'], feed_dict)

        final_preds = [np.expand_dims(pred, 1) for pred in final_preds]
        final_preds = np.concatenate(final_preds, axis = 1)
        
        final_preds = final_preds[:, 0, 0]
        test_y = test_y[:, 0, 0]
        
        MSE_loss = np.mean((final_preds - test_y)**2)
        print("Test mse is: ", MSE_loss)
        return (final_preds, test_y, MSE_loss)


In [ ]:
train_ds = ["CS107Autumn2017dataset.csv", "CS107Autumn2018dataset.csv", "CS107Spring2017dataset.csv", "CS107Spring2018dataset.csv", ]
test_ds = ["CS107Winter2018dataset.csv"]
dir_path = "DeepQueueLearning/Datasets/"
feats_used = ["day", "hourOfDay", "weekNum", "daysAfterPrevAssnDue", "daysUntilNextAssnDue", "daysTilExam", 
                         "isFirstOHWithinLastThreeHour", "NumStudents", "InstructorRating", "AvgHrsSpent"]
output_class = ["loadInflux"]

train_paths, test_paths = [], []
for ds in train_ds:
    train_paths.append(dir_path + ds)
for ds in test_ds:
    test_paths.append(dir_path + ds)
    
sys_model = LSTM_model(feats_used, output_class)
#Train time
tf.reset_default_graph()

with tf.Session() as sess:
    train_preds, train_targets, train_losses = sys_model.train(sess, train_paths)

plt.plot(range(len(train_targets)), train_targets, 'b.', range(len(train_preds)), train_preds, 'r.')
plt.title("Train set ground truths(b.) vs predictions(r.)")
plt.show()

tf.reset_default_graph()
with tf.Session() as sess:
    final_preds, test_y, test_loss = sys_model.test(sess, test_paths)

plt.plot(range(len(test_y)), test_y, 'b.', range(len(test_y)), final_preds, 'r.')
plt.title("Test set ground truths(b.) vs predictions(r.)")
plt.show()

In [ ]:
################## Github model (DO NOT EDIT)####################################

# from tensorflow.contrib import rnn
# from tensorflow.python.ops import variable_scope
# from tensorflow.python.framework import dtypes
# import tensorflow as tf
# import copy
# import os
# import matplotlib.pyplot as plt

# feats_used = ["day", "hourOfDay", "weekNum", "daysAfterPrevAssnDue", "daysUntilNextAssnDue", "daysTilExam", 
#                          "isFirstOHWithinLastThreeHour", "NumStudents", "InstructorRating", "AvgHrsSpent"]
# output_class = ["loadInflux"]

# def get_data(paths):
            
#     frames_X, frames_Y = [], []
#     for path in paths:
#         frames_X.append(pd.read_csv(path, usecols = feats_used))
#         frames_Y.append(pd.read_csv(path, usecols = output_class))
#     return (pd.concat(frames_X), pd.concat(frames_Y))
    
# def generate_train_samples(x, y, input_seq_len, output_seq_len, batch_size = 16):

#     total_start_points = len(x) - input_seq_len - output_seq_len
#     start_x_idx = np.random.choice(range(total_start_points), batch_size, replace = False)

#     input_batch_idxs = [list(range(i, i+input_seq_len)) for i in start_x_idx]
#     input_seq = np.take(x, input_batch_idxs, axis = 0)

#     output_batch_idxs = [list(range(i+input_seq_len, i+input_seq_len+output_seq_len)) for i in start_x_idx]
#     output_seq = np.take(y, output_batch_idxs, axis = 0)

#     return (input_seq, output_seq) # in shape: (batch_size, time_steps, feature_dim)

# def generate_test_samples(x, y, input_seq_len, output_seq_len):

#     total_samples = x.shape[0]

#     input_batch_idxs = [list(range(i, i+input_seq_len)) for i in range((total_samples-input_seq_len-output_seq_len))]
#     input_seq = np.take(x, input_batch_idxs, axis = 0)

#     output_batch_idxs = [list(range(i+input_seq_len, i+input_seq_len+output_seq_len)) for i in range((total_samples-input_seq_len-output_seq_len))]
#     output_seq = np.take(y, output_batch_idxs, axis = 0)

#     return (input_seq, output_seq)

# train_ds = ["CS107Autumn2017dataset.csv", "CS107Autumn2018dataset.csv", "CS107Spring2017dataset.csv",
#             "CS107Spring2018dataset.csv"]
# dir_path = "DeepQueueLearning/Datasets/"
# train_paths, test_paths = [], []
# for ds in train_ds:
#     train_paths.append(dir_path + ds)
    
# X, Y = get_data(train_paths) #All train data in one series

# X_train, Y_train = X.values, Y.values

# ## Parameters
# learning_rate = 0.001
# lambda_l2_reg = 0.01  

# ## Network Parameters
# # length of input signals
# input_seq_len = 10
# # length of output signals
# output_seq_len = 1
# # size of LSTM Cell
# hidden_dim = 512
# # num of input signals
# input_dim = X_train.shape[1]
# # num of output signals
# output_dim = Y_train.shape[1]
# # num of stacked lstm layers 
# num_stacked_layers = 2 
# # gradient clipping - to avoid gradient exploding
# GRADIENT_CLIPPING = 2.5 

# total_iterations = 5000
# batch_size = 512

# def build_graph(feed_previous = False):
    
#     tf.reset_default_graph()
    
#     global_step = tf.Variable(
#                   initial_value=0,
#                   name="global_step",
#                   trainable=False,
#                   collections=[tf.GraphKeys.GLOBAL_STEP, tf.GraphKeys.GLOBAL_VARIABLES])
    
#     weights = {
#         'out': tf.get_variable('Weights_out', \
#                                shape = [hidden_dim, output_dim], \
#                                dtype = tf.float32, \
#                                initializer = tf.truncated_normal_initializer()),
#     }
#     biases = {
#         'out': tf.get_variable('Biases_out', \
#                                shape = [output_dim], \
#                                dtype = tf.float32, \
#                                initializer = tf.constant_initializer(0.)),
#     }
                                          
#     with tf.variable_scope('Seq2seq'):
#         # Encoder: inputs
#         enc_inp = [
#             tf.placeholder(tf.float32, shape=(None, input_dim), name="inp_{}".format(t))
#                for t in range(input_seq_len)
#         ]

#         # Decoder: target outputs
#         target_seq = [
#             tf.placeholder(tf.float32, shape=(None, output_dim), name="y".format(t))
#               for t in range(output_seq_len)
#         ]

#         # Give a "GO" token to the decoder. 
#         # If dec_inp are fed into decoder as inputs, this is 'guided' training; otherwise only the 
#         # first element will be fed as decoder input which is then 'un-guided'
#         dec_inp = [ tf.zeros_like(target_seq[0], dtype=tf.float32, name="GO") ] + target_seq[:-1]

#         with tf.variable_scope('LSTMCell'): 
#             cells = []
#             for i in range(num_stacked_layers):
#                 with tf.variable_scope('RNN_{}'.format(i)):
#                     cells.append(tf.contrib.rnn.LSTMCell(hidden_dim))
#             cell = tf.contrib.rnn.MultiRNNCell(cells)
         
#         def _rnn_decoder(decoder_inputs,
#                         initial_state,
#                         cell,
#                         loop_function=None,
#                         scope=None):
          
#             with variable_scope.variable_scope(scope or "rnn_decoder"):
#                 state = initial_state
#                 outputs = []
#                 prev = None
#                 for i, inp in enumerate(decoder_inputs):
#                     if loop_function is not None and prev is not None:
#                         with variable_scope.variable_scope("loop_function", reuse=True):
#                             inp = loop_function(prev, i)
#                     if i > 0:
#                         variable_scope.get_variable_scope().reuse_variables()
#                     output, state = cell(inp, state)
#                     outputs.append(output)
#                     if loop_function is not None:
#                         prev = output

#             return outputs, state

#         def _basic_rnn_seq2seq(encoder_inputs,
#                               decoder_inputs,
#                               cell,
#                               feed_previous,
#                               dtype=dtypes.float32,
#                               scope=None):
            
#             with variable_scope.variable_scope(scope or "basic_rnn_seq2seq"):
#                 enc_cell = copy.deepcopy(cell)
#                 _, enc_state = rnn.static_rnn(enc_cell, encoder_inputs, dtype=dtype)
#                 if feed_previous:
#                     return _rnn_decoder(decoder_inputs, enc_state, cell, _loop_function)
#                 else:
#                     return _rnn_decoder(decoder_inputs, enc_state, cell)

#         def _loop_function(prev, _):
#             return tf.matmul(prev, weights['out']) + biases['out']
        
#         dec_outputs, dec_memory = _basic_rnn_seq2seq(
#             enc_inp, 
#             dec_inp, 
#             cell, 
#             feed_previous = feed_previous
#         )
# #         print(dec_outputs)
#         reshaped_outputs = [tf.matmul(i, weights['out']) + biases['out'] for i in dec_outputs]
        
#     # Training loss and optimizer
#     with tf.variable_scope('Loss'):
#         # L2 loss
#         output_loss = 0
#         for _y, _Y in zip(reshaped_outputs, target_seq):
#             output_loss += tf.reduce_mean(tf.pow(_y - _Y, 2))

#         # L2 regularization for weights and biases
#         reg_loss = 0
#         for tf_var in tf.trainable_variables():
#             if 'Biases_' in tf_var.name or 'Weights_' in tf_var.name:
#                 reg_loss += tf.reduce_mean(tf.nn.l2_loss(tf_var))

#         loss = output_loss + lambda_l2_reg * reg_loss

#     with tf.variable_scope('Optimizer'):
#         optimizer = tf.contrib.layers.optimize_loss(
#                 loss=loss,
#                 learning_rate=learning_rate,
#                 global_step=global_step,
#                 optimizer='Adam',
#                 clip_gradients=GRADIENT_CLIPPING)
        
#     saver = tf.train.Saver
    
#     return dict(
#         enc_inp = enc_inp, 
#         target_seq = target_seq, 
#         train_op = optimizer, 
#         loss=loss,
#         saver = saver, 
#         reshaped_outputs = reshaped_outputs,
#         )

# train_losses = []
# val_losses = []

# rnn_model = build_graph(feed_previous=False)

# saver = tf.train.Saver()

# init = tf.global_variables_initializer()
# train_preds, train_targets = [], []
# with tf.Session() as sess:

#     sess.run(init)

#     try:
#         for i in range(total_iterations):
#             batch_input, batch_output = generate_train_samples(X_train, Y_train, input_seq_len, output_seq_len,
#                                                                batch_size=batch_size)
#             print(batch_input.shape)
#             feed_dict = {rnn_model['enc_inp'][t]: batch_input[:,t] for t in range(input_seq_len)}
#             feed_dict.update({rnn_model['target_seq'][t]: batch_output[:,t] for t in range(output_seq_len)})
#             _, loss_t, reshaped_outputs, target_seq = sess.run([rnn_model['train_op'], rnn_model['loss'],
#                                                            rnn_model['reshaped_outputs'], rnn_model['target_seq']],
#                                                           feed_dict)
#             print("Training iteration: {}, Loss: {}".format(i+1, loss_t))
# #             print(type(train_preds), type(target_seq))
#             if (i % 10 == 0) : 
#                 train_losses.append(loss_t)
#                 for i in range(len(target_seq)):
#                     train_preds.append(reshaped_outputs[i]) 
#                     train_targets.append(target_seq[i]) 
#     except KeyboardInterrupt:
#         print("Training interrupted.")
        
#     temp_saver = rnn_model['saver']()
#     save_path = temp_saver.save(sess, os.path.join('./saved_model/', 'multivariate_ts_pollution_case'))
#     plt.plot(train_losses) 
# print("Checkpoint saved at: ", save_path)